# Machine Learning Foundation

## Course 5, Part g: Transfer Learning DEMO

For this exercise, we will use the well-known MNIST digit data. To illustrate the power and concept of transfer learning, we will train a CNN on just the digits 5,6,7,8,9.  Then we will train just the last layer(s) of the network on the digits 0,1,2,3,4 and see how well the features learned on 5-9 help with classifying 0-4.

Adapted from https://github.com/fchollet/keras/blob/master/examples/mnist_transfer_cnn.py

In [1]:
import datetime
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
#from tensorflow import keras
#from tensorflow.keras.datasets import mnist
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
#from tensorflow.keras.layers import Conv2D, MaxPooling2D
#from tensorflow.keras import backend as K

In [2]:
#used to help some of the timing functions
now = datetime.datetime.now

In [3]:
# set some parameters
batch_size = 128
num_classes = 5
epochs = 5

In [4]:
# set some more parameters
img_rows, img_cols = 28, 28
filters = 32
pool_size = 2
kernel_size = 3

In [5]:
## This just handles some variability in how the input data is loaded

if K.image_data_format() == 'channels_first':
    input_shape = (1, img_rows, img_cols)
else:
    input_shape = (img_rows, img_cols, 1)

In [6]:
## To simplify things, write a function to include all the training steps
## As input, function takes a model, training set, test set, and the number of classes
## Inside the model object will be the state about which layers we are freezing and which we are training

def train_model(model, train, test, num_classes):
    x_train = train[0].reshape((train[0].shape[0],) + input_shape)
    x_test = test[0].reshape((test[0].shape[0],) + input_shape)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(train[1], num_classes)
    y_test = keras.utils.to_categorical(test[1], num_classes)

    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])

    t = now()
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test))
    print('Training time: %s' % (now() - t))

    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])

In [7]:
# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# create two datasets: one with digits below 5 and one with 5 and above
x_train_lt5 = x_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
x_test_lt5 = x_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

x_train_gte5 = x_train[y_train >= 5]
y_train_gte5 = y_train[y_train >= 5] - 5
x_test_gte5 = x_test[y_test >= 5]
y_test_gte5 = y_test[y_test >= 5] - 5

In [8]:
# Define the "feature" layers.  These are the early layers that we expect will "transfer"
# to a new problem.  We will freeze these layers during the fine-tuning process

feature_layers = [
    Conv2D(filters, kernel_size,
           padding='valid',
           input_shape=input_shape),
    Activation('relu'),
    Conv2D(filters, kernel_size),
    Activation('relu'),
    MaxPooling2D(pool_size=pool_size),
    Dropout(0.25),
    Flatten(),
]

In [9]:
# Define the "classification" layers.  These are the later layers that predict the specific classes from the features
# learned by the feature layers.  This is the part of the model that needs to be re-trained for a new problem

classification_layers = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(num_classes),
    Activation('softmax')
]

In [10]:
# We create our model by combining the two sets of layers as follows
model = Sequential(feature_layers + classification_layers)

Metal device set to: Apple M2

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2023-03-20 14:57:54.051693: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-03-20 14:57:54.051820: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [11]:
# Let's take a look
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 activation (Activation)     (None, 26, 26, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 24, 24, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 12, 12, 32)        0         
                                                        

In [12]:
# Now, let's train our model on the digits 5,6,7,8,9

train_model(model,
            (x_train_gte5, y_train_gte5),
            (x_test_gte5, y_test_gte5), num_classes)

x_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples
Epoch 1/5


2023-03-20 14:57:54.356594: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-03-20 14:57:54.602778: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


230/230 [==============================] - ETA: 0s - loss: 1.6101 - accuracy: 0.2025

2023-03-20 14:57:59.445984: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


230/230 [==============================] - 5s 21ms/step - loss: 1.6101 - accuracy: 0.2025 - val_loss: 1.5869 - val_accuracy: 0.2631
Epoch 2/5
230/230 [==============================] - 4s 18ms/step - loss: 1.5814 - accuracy: 0.2679 - val_loss: 1.5555 - val_accuracy: 0.3917
Epoch 3/5
230/230 [==============================] - 4s 18ms/step - loss: 1.5514 - accuracy: 0.3390 - val_loss: 1.5217 - val_accuracy: 0.5682
Epoch 4/5
230/230 [==============================] - 4s 18ms/step - loss: 1.5193 - accuracy: 0.4092 - val_loss: 1.4842 - val_accuracy: 0.6780
Epoch 5/5
230/230 [==============================] - 4s 18ms/step - loss: 1.4843 - accuracy: 0.4796 - val_loss: 1.4423 - val_accuracy: 0.7322
Training time: 0:00:22.008610
Test score: 1.4422764778137207
Test accuracy: 0.7321538925170898


### Freezing Layers
Keras allows layers to be "frozen" during the training process.  That is, some layers would have their weights updated during the training process, while others would not.  This is a core part of transfer learning, the ability to train just the last one or several layers.

Note also, that a lot of the training time is spent "back-propagating" the gradients back to the first layer.  Therefore, if we only need to compute the gradients back a small number of layers, the training time is much quicker per iteration.  This is in addition to the savings gained by being able to train on a smaller data set.

In [13]:
# Freeze only the feature layers
for l in feature_layers:
    l.trainable = False

Observe below the differences between the number of *total params*, *trainable params*, and *non-trainable params*.

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 activation (Activation)     (None, 26, 26, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 24, 24, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 12, 12, 32)        0         
                                                        

In [15]:
train_model(model,
            (x_train_lt5, y_train_lt5),
            (x_test_lt5, y_test_lt5), num_classes)

x_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples
Epoch 1/5
 12/240 [>.............................] - ETA: 2s - loss: 1.5959 - accuracy: 0.2533

2023-03-20 14:58:17.487142: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


240/240 [==============================] - ETA: 0s - loss: 1.5643 - accuracy: 0.3006

2023-03-20 14:58:19.905654: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


240/240 [==============================] - 3s 11ms/step - loss: 1.5643 - accuracy: 0.3006 - val_loss: 1.5203 - val_accuracy: 0.4557
Epoch 2/5
240/240 [==============================] - 3s 11ms/step - loss: 1.5088 - accuracy: 0.3804 - val_loss: 1.4599 - val_accuracy: 0.5608
Epoch 3/5
240/240 [==============================] - 3s 11ms/step - loss: 1.4529 - accuracy: 0.4717 - val_loss: 1.4002 - val_accuracy: 0.6984
Epoch 4/5
240/240 [==============================] - 3s 11ms/step - loss: 1.3986 - accuracy: 0.5592 - val_loss: 1.3418 - val_accuracy: 0.7861
Epoch 5/5
240/240 [==============================] - 3s 11ms/step - loss: 1.3472 - accuracy: 0.6349 - val_loss: 1.2854 - val_accuracy: 0.8406
Training time: 0:00:13.091923
Test score: 1.2853822708129883
Test accuracy: 0.8406304717063904


Note that after a single epoch, we are already achieving results on classifying 0-4 that are comparable to those achieved on 5-9 after 5 full epochs.  This despite the fact the we are only "fine-tuning" the last layer of the network, and all the early layers have never seen what the digits 0-4 look like.

Also, note that even though nearly all (590K/600K) of the *parameters* were trainable, the training time per epoch was still much reduced.  This is because the unfrozen part of the network was very shallow, making backpropagation faster. 

## Exercise
- Now we will write code to reverse this training process.  That is, train on the digits 0-4, then finetune only the last layers on the digits 5-9.

In [16]:
# Create layers and define the model as above
feature_layers2 = [
    Conv2D(filters, kernel_size,
           padding='valid',
           input_shape=input_shape),
    Activation('relu'),
    Conv2D(filters, kernel_size),
    Activation('relu'),
    MaxPooling2D(pool_size=pool_size),
    Dropout(0.25),
    Flatten(),
]

classification_layers2 = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(num_classes),
    Activation('softmax')
]
model2 = Sequential(feature_layers2 + classification_layers2)
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 activation_4 (Activation)   (None, 26, 26, 32)        0         
                                                                 
 conv2d_3 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 activation_5 (Activation)   (None, 24, 24, 32)        0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 12, 12, 32)       0         
 2D)                                                             
                                                                 
 dropout_2 (Dropout)         (None, 12, 12, 32)        0         
                                                      

In [17]:
# Now, let's train our model on the digits 0,1,2,3,4
train_model(model2,
            (x_train_lt5, y_train_lt5),
            (x_test_lt5, y_test_lt5), num_classes)

x_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples
Epoch 1/5
  4/240 [..............................] - ETA: 4s - loss: 1.6103 - accuracy: 0.1699  

2023-03-20 14:58:31.631641: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


240/240 [==============================] - ETA: 0s - loss: 1.5982 - accuracy: 0.2191

2023-03-20 14:58:36.054415: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


240/240 [==============================] - 5s 19ms/step - loss: 1.5982 - accuracy: 0.2191 - val_loss: 1.5745 - val_accuracy: 0.3217
Epoch 2/5
240/240 [==============================] - 4s 18ms/step - loss: 1.5660 - accuracy: 0.3010 - val_loss: 1.5390 - val_accuracy: 0.5182
Epoch 3/5
240/240 [==============================] - 4s 18ms/step - loss: 1.5319 - accuracy: 0.3889 - val_loss: 1.4998 - val_accuracy: 0.6634
Epoch 4/5
240/240 [==============================] - 4s 19ms/step - loss: 1.4924 - accuracy: 0.4761 - val_loss: 1.4545 - val_accuracy: 0.7826
Epoch 5/5
240/240 [==============================] - 4s 18ms/step - loss: 1.4475 - accuracy: 0.5578 - val_loss: 1.3992 - val_accuracy: 0.8554
Training time: 0:00:22.509787
Test score: 1.3992493152618408
Test accuracy: 0.855419397354126


In [18]:
#Freeze layers
for l in feature_layers2:
    l.trainable = False

In [19]:
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 activation_4 (Activation)   (None, 26, 26, 32)        0         
                                                                 
 conv2d_3 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 activation_5 (Activation)   (None, 24, 24, 32)        0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 12, 12, 32)       0         
 2D)                                                             
                                                                 
 dropout_2 (Dropout)         (None, 12, 12, 32)        0         
                                                      

In [20]:
train_model(model2,
            (x_train_gte5, y_train_gte5),
            (x_test_gte5, y_test_gte5), num_classes)

x_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples
Epoch 1/5
  3/230 [..............................] - ETA: 6s - loss: 1.6138 - accuracy: 0.2500 

2023-03-20 14:58:55.187859: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


230/230 [==============================] - ETA: 0s - loss: 1.5969 - accuracy: 0.2756

2023-03-20 14:58:57.577842: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


230/230 [==============================] - 3s 11ms/step - loss: 1.5969 - accuracy: 0.2756 - val_loss: 1.5710 - val_accuracy: 0.4174
Epoch 2/5
230/230 [==============================] - 2s 10ms/step - loss: 1.5716 - accuracy: 0.3123 - val_loss: 1.5421 - val_accuracy: 0.4388
Epoch 3/5
230/230 [==============================] - 2s 10ms/step - loss: 1.5436 - accuracy: 0.3571 - val_loss: 1.5138 - val_accuracy: 0.4653
Epoch 4/5
230/230 [==============================] - 2s 10ms/step - loss: 1.5177 - accuracy: 0.4045 - val_loss: 1.4860 - val_accuracy: 0.5151
Epoch 5/5
230/230 [==============================] - 2s 10ms/step - loss: 1.4918 - accuracy: 0.4553 - val_loss: 1.4583 - val_accuracy: 0.5822
Training time: 0:00:12.332671
Test score: 1.4583102464675903
Test accuracy: 0.5821847319602966


In [21]:
import tensorflow as tf

cifar = tf.keras.datasets.cifar100
(x_train, y_train), (x_test, y_test) = cifar.load_data()
model = tf.keras.applications.ResNet50(
    include_top=True,
    weights=None,
    input_shape=(32, 32, 3),
    classes=100,)

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer="adam", loss=loss_fn, metrics=["accuracy"])
model.fit(x_train, y_train, epochs=5, batch_size=64)

169001437/169001437 [==============================] - 7s 0us/step
Epoch 1/5


/Users/itnas/miniconda3/envs/tensorflow/lib/python3.10/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)
2023-03-20 15:00:08.332582: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


782/782 [==============================] - 89s 103ms/step - loss: 4.8630 - accuracy: 0.0646
Epoch 2/5
782/782 [==============================] - 78s 100ms/step - loss: 4.2030 - accuracy: 0.1212
Epoch 3/5
782/782 [==============================] - 78s 100ms/step - loss: 4.1264 - accuracy: 0.1254
Epoch 4/5
782/782 [==============================] - 78s 100ms/step - loss: 3.7094 - accuracy: 0.1699
Epoch 5/5
782/782 [==============================] - 80s 102ms/step - loss: 3.7753 - accuracy: 0.1746


---
### Machine Learning Foundation (C) 2020 IBM Corporation